## Scraper Fundamentus

Scraper para ler/coletar automaticamente dados de todas as empresas listadas no site http://fundamentus.com.br/, viabilizando uma análise mais automatizada dos valores fundamentalista destas empresas.

In [ ]:
# instala bibliotecas externas
!pip install requests
!pip install lxml

# import
import re
import requests
import numpy as np
import pandas as pd

from tqdm import tqdm
from datetime import date, timedelta, datetime
from lxml.html import fragment_fromstring

Função para conversão de valores numéricos apresentados no site:

In [2]:
# converte valores para numéricos (quando assim o são)
def _convert_data(data):
    data = data.strip()
    try:
        data_float = data.replace('.', '').replace(',', '.')
        # convert 10.5% to 0.105 float value
        if data_float.endswith('%'):
            data_float = data_float[:-1]
            return float(data_float) / 100
        return float(data_float)
    except ValueError:
        return data

Função para coletar os valores genéricos das empresas listadas:
> O filtro `negociada: ON` lista somente as ações que tiveram negociação nos últimos 2 meses.  
> Remova este filtro, e o site disponibilizará tickers que já não são mais negociados na B3 (como por exemplo: VALE5).

In [3]:
# carrega valores apresentados na página que lista todos os dados da empresa
def load_generic_data():
    post_filter = {'negociada': 'ON'}  # only traded stocks (on the last 2 months)
    html_data = requests.post('http://www.fundamentus.com.br/resultado.php', data = post_filter)

    pattern = re.compile('<table id="resultado".*</table>', re.DOTALL)
    [table] = re.findall(pattern, html_data.text)
    page = fragment_fromstring(table)

    [thead] = page.xpath('thead')
    [tr] = thead.xpath('tr')
    headers = [th.text_content().strip() for th in tr.xpath('th')]

    [tbody] = page.xpath('tbody')

    stock_info = {}

    for tr in tbody.xpath('tr'):
        data = [_convert_data(i.text_content().strip()) for i in tr.xpath('td')]
        stock_data = dict(zip(headers, data))
        tick = stock_data['Papel']
        stock_info[tick] = stock_data

    return stock_info

In [4]:
generic_data = load_generic_data()
generic_df = pd.DataFrame.from_dict(generic_data, orient='index')

In [5]:
# verificação da leitura (valores Div.Yield, ROIC e ROE, por exemplo, são valores percentuais)
show_columns = ['Cotação', 'P/L', 'P/VP', 'Div.Yield', 'P/EBIT', 'EV/EBIT', 'ROIC', 'ROE']
generic_df.sort_index().head()[show_columns]

,Cotação,P/L,P/VP,Div.Yield,P/EBIT,EV/EBIT,ROIC,ROE
AALR3,10.64,30.48,0.98,0.0079,10.01,14.19,0.0557,0.0322
ABCB4,13.60,5.62,0.73,0.0772,0.00,0.00,0.0000,0.1308
ABEV3,12.10,16.16,3.11,0.0405,12.21,11.64,0.2061,0.1922
ADHM3,3.35,-4.07,-1.28,0.0000,-5.67,-5.67,5.2170,0.3147
AFLT3,6.85,17.11,2.02,0.0154,17.31,15.29,0.1456,0.1178


Função para coletar os valores específicos de cada empresa:

In [6]:
# carrega valores apresentados na página individual de cada empresa
def get_specific_data(ticker):
    html_data = requests.get(f'http://www.fundamentus.com.br/detalhes.php?papel={ticker}')

    [table_section] = re.findall(re.compile('<table.*</table>', re.DOTALL), html_data.text)
    tables = [
        fragment_fromstring(t + '</table>')
        for t in table_section.split('</table>')
        if t.strip()
    ]

    data = {}

    for table in tables:
        for tr in table.xpath('tr'):
            td_label = [
                i.xpath('span')[-1].text_content().strip()
                for i in tr.xpath('td')
                if 'label' in i.attrib.get('class')
            ]

            td_data = [
                _convert_data(i.xpath('span')[-1].text_content().strip())
                for i in tr.xpath('td')
                if 'data' in i.attrib.get('class')
            ]

            while td_label:
                key = td_label.pop(0)
                value = td_data.pop(0)

                # create new name for duplicate columns
                # avoiding data replacement
                if key:
                    while key in data:
                        key += '_'
                    data[key] = value

    return data

Coleta as informações de cada ação/ticker listada/o no site:

In [7]:
specific_data = {}

# Pega lista das ações listadas
tickers = generic_df.index
# tickers = ['ITSA4', 'VALE3', 'PETR4']

for ticker in tqdm(generic_df.index):
    specific_data[ticker] = get_specific_data(ticker)

specific_df = pd.DataFrame.from_dict(specific_data, orient='index')

100%|██████████| 409/409 [05:38<00:00,  1.21it/s]


In [8]:
# verificação da leitura (valores Div. Yield, ROIC e ROE, por exemplo, são valores percentuais)
show_columns = ['Cotação', 'P/L', 'P/VP', 'Div. Yield', 'P/EBIT', 'EV / EBIT', 'ROIC', 'ROE']
specific_df.sort_index().head()[show_columns]

,Cotação,P/L,P/VP,Div. Yield,P/EBIT,EV / EBIT,ROIC,ROE
AALR3,10.64,30.48,0.98,0.008,10.01,14.19,0.056,0.032
ABCB4,13.60,5.62,0.73,0.077,-,-,-,0.131
ABEV3,12.10,16.16,3.11,0.041,12.21,11.64,0.206,0.192
ADHM3,3.35,-4.07,-1.28,0.000,-5.67,-5.67,5.217,0.315
AFLT3,6.85,17.11,2.02,0.015,17.31,15.29,0.146,0.118


Lista de colunas disponíveis:
> É possível ver na lista que há colunas 'duplicadas' referente `Dados demonstrativos de resultados`, sendo as primeiras colunas referente dados de `12 meses` (*Receita Líquida*, *EBIT* e *Lucro Líquido*) e as outras referente dados de `3 meses` (*Receita Líquida_*, *EBIT_* e *Lucro Líquido_* - com '_' no final)

In [9]:
specific_df.columns

Index(['Papel', 'Cotação', 'Tipo', 'Data últ cot', 'Empresa', 'Min 52 sem',
       'Setor', 'Max 52 sem', 'Subsetor', 'Vol $ méd (2m)', 'Valor de mercado',
       'Últ balanço processado', 'Valor da firma', 'Nro. Ações', 'Dia', 'P/L',
       'LPA', 'Mês', 'P/VP', 'VPA', '30 dias', 'P/EBIT', 'Marg. Bruta',
       '12 meses', 'PSR', 'Marg. EBIT', '2020', 'P/Ativos', 'Marg. Líquida',
       '2019', 'P/Cap. Giro', 'EBIT / Ativo', '2018', 'P/Ativ Circ Liq',
       'ROIC', '2017', 'Div. Yield', 'ROE', '2016', 'EV / EBITDA',
       'Liquidez Corr', '2015', 'EV / EBIT', 'Div Br/ Patrim',
       'Cres. Rec (5a)', 'Giro Ativos', 'Ativo', 'Dív. Bruta',
       'Disponibilidades', 'Dív. Líquida', 'Ativo Circulante', 'Patrim. Líq',
       'Receita Líquida', 'Receita Líquida_', 'EBIT', 'EBIT_', 'Lucro Líquido',
       'Lucro Líquido_', 'Depósitos', 'Cart. de Crédito', 'Result Int Financ',
       'Result Int Financ_', 'Rec Serviços', 'Rec Serviços_'],
      dtype='object')

Limpa dados `-` dos dados:

In [10]:
specific_df = specific_df.replace('-', np.nan)

Exporta dados para arquivo (ordenados):

In [18]:
sorted_df = specific_df

# ordena por nome da colunas (axis=y)
# sorted_df = specific_df[sorted(specific_df.columns)]

# ordena por nome do ativo (axis=x)
sorted_df = sorted_df.sort_index()

# exporta para arquivo formato csv
sorted_df.to_csv('fundamentus.csv')